# Modeling relationships with referential constraints
## DataJoint example

In [1]:
import datajoint as dj

In [2]:
schema = dj.schema('dimitri_language')

[2022-09-06 18:01:07,421][INFO]: Connecting dimitri@db.ust-data-sci.net:3306
[2022-09-06 18:01:07,723][INFO]: Connected dimitri@db.ust-data-sci.net:3306


In [ ]:
@schema
class Title(dj.Lookup):
    definition = """
    title_code  : char(8)
    ---
    full_title : char(120)
    """
    
    contents = [
        ("SW-Dev1", "Software Developer 1"),
        ("SW-Dev2", "Software Developer 2"),
        ("SW-Dev3", "Software Developer 3"),
        ("Web-Dev1", "Web Developer 1"),
        ("Web-Dev2", "Web Developer 2"),
        ("Web-Dev3", "Web Developer 3"),
        ("HR-Mgr", "Human Resources Manager")
    ]

In [ ]:
@schema
class Person(dj.Manual):
    definition = """
    person_id  :  int 
    ---
    first_name : varchar(30)
    last_name : varchar(30)
    -> Title
    """

In [ ]:
Title()

In [ ]:
Person().insert1((1, "Jane", "Doe", "HR-Mgr"))

In [ ]:
Person().insert1((2, "Johnny", "Cash", "Web-Dev1"))

In [ ]:
Person()

In [ ]:
Title().delete()

In [ ]:
Title & {'title_code': "HR-Mgr"}

In [ ]:
(Title & {'title_code': "Web-Dev2"}).delete()

In [ ]:
@schema
class Retirement(dj.Manual):
    definition = """
    -> Person
    ---
    retirement_date : date
    """

In [ ]:
Retirement()

In [ ]:
Title.drop()

In [ ]:
dj.Diagram(schema)

In [ ]:
@schema 
class Language(dj.Lookup):
    definition = """
    lang_code  : char(8)
    ---
    language : varchar(20)
    """
    
    contents = [
        ("Eng", "English"),
        ("Nav", "Navajo"),
        ("Fr", "French"),
        ("It", "Italian")        
    ]

In [ ]:
Language()

In [ ]:
dj.Diagram(schema)

In [ ]:
@schema
class LanguageSkill(dj.Manual):
    definition = """
    -> Person
    -> Language
    ---
    skill_level  : enum("beginner", "intermediate", "fluent", "native")
    """

In [ ]:
Person()

In [ ]:
dj.Diagram(schema)

In [ ]:
LanguageSkill.insert1((1, "Eng", "native"))

In [ ]:
LanguageSkill.insert1((1, "It", "beginner"))

In [ ]:
LanguageSkill.insert1((1, "Nav", "fluent"))

In [ ]:
LanguageSkill.insert1((2, "Fr", "fluent"))

In [ ]:
LanguageSkill.insert1((2, "It", "intermediate"))

In [ ]:
LanguageSkill()

In [ ]:
LanguageSkill.fetch(order_by="skill_level", format="frame")

# Now in SQL

In [ ]:
# prepare SQL Magic
import json
with open('cred.json') as f:
    creds = json.load(f)
    
connection_string = "mysql://{user}:{password}@{host}".format(**creds)

import pymysql 
pymysql.install_as_MySQLdb()

In [ ]:
%load_ext sql
%config SqlMagic.autocommit=True

In [ ]:
%sql $connection_string

In [ ]:
%%sql

SHOW TABLES IN dimitri_language;

In [ ]:
%%sql 

SHOW CREATE TABLE dimitri_language.person;

```sql
CREATE TABLE dimitri_language.person (
   person_id int(11) NOT NULL,
   first_name varchar(30) NOT NULL,
   last_name varchar(30) NOT NULL,
   title_code char(8) NOT NULL,
   PRIMARY KEY (person_id),
   FOREIGN KEY (title_code) REFERENCES title (title_code)
)
```

In [ ]:
%%sql

SHOW CREATE TABLE dimitri_language.language_skill;

```sql
CREATE TABLE dimitri_language.language_skill (
   person_id int(11) NOT NULL,
   lang_code char(8) NOT NULL,
   skill_level enum('beginner','intermediate','fluent','native') NOT NULL,
   PRIMARY KEY (person_id, lang_code),
   FOREIGN KEY (person_id) REFERENCES person (person_id),
   FOREIGN KEY (lang_code) REFERENCES language (lang_code)
)
```

Contast with the DataJoint definition:
```python
@schema
class LanguageSkill(dj.Manual):
    definition = """
    -> Person
    -> Language
    ---
    skill_level  : enum("beginner", "intermediate", "fluent", "native")
    """
```

# Now in SQL from Python

In [ ]:
import pymysql

In [ ]:
conn = pymysql.connect(...)
cursor = conn.cursor(...)

In [ ]:
cursor.execute("""
CREATE TABLE language_skill (
   person_id int(11) NOT NULL,
   lang_code char(8) NOT NULL,
   skill_level enum('beginner','intermediate','fluent','native') NOT NULL,
   PRIMARY KEY (person_id, lang_code),
   FOREIGN KEY (person_id) REFERENCES person (person_id),
   FOREIGN KEY (lang_code) REFERENCES language (lang_code)
) 
"""
)